In [4]:
import os
import pandas as pd
import statsmodels.api as sm

In [67]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

In [6]:
market = pd.read_csv(os.path.join('resources', 'cleaned_data.csv')).drop(columns=['Unnamed: 0'])

In [27]:
indicators = pd.DataFrame({
   'Indicator': market.columns.tolist()
})

filt = indicators[indicators['Indicator'].str.contains('Growth|growth|Margin|ratio|Ratio|Turnover|per|/| / | to |Per|return|To|days|Days') == False]

cond = filt['Indicator'].to_list()

# LOGISTIC CLASSIFICATION MODELS

## Full Dataset: 200+ predictors

In [77]:
y = market['Class']

In [78]:
x = market.drop(columns={'Ticker','Company Name','2019 PRICE VAR [%]','Class','Market Cap Size'})

X = pd.get_dummies(x)

X.head()

,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,...,Sector_Communication Services,Sector_Consumer Cyclical,Sector_Consumer Defensive,Sector_Energy,Sector_Financial Services,Sector_Healthcare,Sector_Industrials,Sector_Real Estate,Sector_Technology,Sector_Utilities
0,15926000.0,-0.1230,0.0,15926000.0,0.0,22534000.0,22534000.0,-6608000.0,0.0,-10501000.0,...,0,0,0,0,1,0,0,0,0,0
1,185553000.0,0.0245,132650000.0,52903000.0,0.0,39042000.0,51837000.0,1066000.0,2037000.0,-971000.0,...,0,0,0,0,1,0,0,0,0,0
2,267465000.0,-0.3004,179692000.0,87773000.0,49903000.0,48638000.0,98541000.0,-10768000.0,1106000.0,-9778000.0,...,0,0,0,0,0,0,0,0,1,0
3,433947000.0,0.0709,330414000.0,103533000.0,0.0,47755000.0,47743000.0,55790000.0,0.0,55939000.0,...,0,0,0,0,0,0,0,0,0,0
4,330867000.0,0.0504,121455000.0,209412000.0,0.0,22784000.0,129877000.0,79535000.0,52248000.0,19686000.0,...,0,0,0,0,0,0,0,1,0,0


In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

classifier = LogisticRegression()

classifier.fit(X_train, y_train)

print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

/Users/danvaldes/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Training Data Score: 0.691559569294894
Testing Data Score: 0.6833333333333333


/Users/danvaldes/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [80]:
y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.45      0.09      0.16       298
           1       0.70      0.95      0.81       662

    accuracy                           0.68       960
   macro avg       0.58      0.52      0.48       960
weighted avg       0.62      0.68      0.60       960



## Feature Selection: KBest method

In [75]:
selector = SelectKBest(f_classif, k=12)

X_new = selector.fit_transform(X, y)

selected_features = pd.DataFrame(selector.inverse_transform(X_new), 
                                 index=X.index, 
                                 columns=X.columns)
selected_features.head()

# Dropped columns have values of all 0s, so var is 0, drop them
selected_columns = selected_features.columns[selected_features.var() != 0]

# Get the valid dataset with the selected features.
X_final = selected_features[selected_columns]

/Users/danvaldes/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [84] are constant.
  UserWarning)
/Users/danvaldes/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [76]:
X_final.head()

,Operating Income,Earnings before Tax,Net Income,Net Income Com,Dividend per Share,EBIT,Consolidated Income,Market Cap,Enterprise Value,5Y Dividend per Share Growth (per Share),Sector_Financial Services,Sector_Healthcare
0,-6608000.0,-10501000.0,-10876000.0,-11082000.0,0.00,-10501000.0,-10876000.0,4.848362e+07,6.546622e+06,0.0000,1.0,0.0
1,1066000.0,-971000.0,-704000.0,-1103000.0,0.02,1066000.0,-704000.0,4.868035e+07,6.978835e+07,0.0000,1.0,0.0
2,-10768000.0,-9778000.0,-2146000.0,-2146000.0,0.00,-8672000.0,-2146000.0,3.054934e+08,3.360344e+08,0.0000,0.0,0.0
3,55790000.0,55939000.0,42572000.0,42572000.0,0.32,55939000.0,42572000.0,1.826434e+09,1.824413e+09,0.2358,0.0,0.0
4,79535000.0,19686000.0,19686000.0,19686000.0,1.09,71934000.0,27202000.0,1.896913e+09,3.130413e+09,0.0510,0.0,0.0


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y, random_state=1, stratify=y)

classifier = LogisticRegression()

classifier.fit(X_train, y_train)

print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

In [28]:
y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.55      0.12      0.20       298
           1       0.71      0.95      0.81       662

    accuracy                           0.70       960
   macro avg       0.63      0.54      0.51       960
weighted avg       0.66      0.70      0.62       960



# LINEAR REGRESSION MODELS

In [11]:
y = market["2019 PRICE VAR [%]"]

In [28]:
x = market[cond].drop(columns={'Ticker','Company Name','2019 PRICE VAR [%]','Class','Market Cap Size'})

X = pd.get_dummies(x)

X.head()

,Revenue,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Interest Expense,Earnings before Tax,Income Tax Expense,Net Income - Non-Controlling int,Net Income - Discontinued ops,...,Sector_Communication Services,Sector_Consumer Cyclical,Sector_Consumer Defensive,Sector_Energy,Sector_Financial Services,Sector_Healthcare,Sector_Industrials,Sector_Real Estate,Sector_Technology,Sector_Utilities
0,15926000.0,0.0,15926000.0,0.0,22534000.0,0.0,-10501000.0,375000.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1,185553000.0,132650000.0,52903000.0,0.0,39042000.0,2037000.0,-971000.0,-267000.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
2,267465000.0,179692000.0,87773000.0,49903000.0,48638000.0,1106000.0,-9778000.0,-7632000.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
3,433947000.0,330414000.0,103533000.0,0.0,47755000.0,0.0,55939000.0,13367000.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,330867000.0,121455000.0,209412000.0,0.0,22784000.0,52248000.0,19686000.0,0.0,7516000.0,0.0,...,0,0,0,0,0,0,0,1,0,0


In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [30]:
reg = sm.OLS(y_train, X_train).fit()

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     2019 PRICE VAR [%]   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     1.604
Date:                Mon, 11 May 2020   Prob (F-statistic):           0.000507
Time:                        20:12:34   Log-Likelihood:                -15647.
No. Observations:                2879   AIC:                         3.146e+04
Df Residuals:                    2795   BIC:                         3.196e+04
Df Model:                          83                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Revenue                          -1.545e-07   1.19e-07     -1.297      0.195   -3.88e-07    7.91e-08
Cost of Revenue                   1.546e-07   1.19e-07      1.298      0.195    -7.9e-08    3.88e-07
Gross Profit                      1.544e-07   1.19e-07      1.296      0.195   -7.92e-08    3.88e-07
R&D Expenses                     -2.375e-09   2.83e-09     -0.841      0.401   -7.91e-09    3.17e-09
SG&A Expense                     -1.353e-10   1.02e-09     -0.133      0.894   -2.13e-09    1.86e-09
Interest Expense                 -1.368e-07   7.59e-08     -1.801      0.072   -2.86e-07    1.21e-08
Earnings before Tax               3.913e-08   4.69e-08      0.835      0.404   -5.28e-08    1.31e-07
Income Tax Expense               -1.771e-07    8.7e-08     -2.035      0.042   -3.48e-07   -6.49e-09
Net Income - Non-Controlling int  9.812e-09   2.16e-08      0.454      0.650   -3.26e-08    5.22e-08
Net Income - Discontinued ops     2.142e-09   1.15e-08      0.186      0.852   -2.04e-08    2.47e-08
Net Income                       -1.104e-07    8.2e-08     -1.347      0.178   -2.71e-07    5.04e-08
Preferred Dividends                -6.3e-08   5.11e-08     -1.232      0.218   -1.63e-07    3.73e-08
Net Income Com                   -5.866e-08   4.64e-08     -1.264      0.206    -1.5e-07    3.23e-08
EPS                                  0.5078      0.097      5.250      0.000       0.318       0.697
EPS Diluted                          0.0003      0.001      0.528      0.598      -0.001       0.001
Weighted Average Shs Out          1.086e-08   4.33e-08      0.251      0.802    -7.4e-08    9.57e-08
Weighted Average Shs Out (Dil)   -1.085e-08   4.37e-08     -0.248      0.804   -9.66e-08    7.49e-08
Free Cash Flow margin               -0.0051      0.010     -0.521      0.603      -0.024       0.014
EBITDA                            1.125e-09   1.14e-08      0.098      0.922   -2.13e-08    2.36e-08
EBIT                              1.391e-07   7.63e-08      1.823      0.068   -1.06e-08    2.89e-07
Consolidated Income              -9.343e-09   2.64e-08     -0.354      0.723   -6.11e-08    4.24e-08
Cash and cash equivalents        -1.259e-10   2.18e-09     -0.058      0.954   -4.41e-09    4.15e-09
Short-term investments           -8.334e-11   2.14e-09     -0.039      0.969   -4.28e-09    4.11e-09
Cash and short-term investments   1.509e-10   2.08e-09      0.072      0.942   -3.93e-09    4.23e-09
Receivables                      -3.766e-09   5.43e-09     -0.693      0.488   -1.44e-08    6.89e-09
Inventories                      -5.006e-10   1.15e-09     -0.436      0.663   -2.75e-09    1.75e-09
Goodwill and Intangible Assets    3.379e-10   3.43e-10      0.986      0.324   -3.34e-10    1.01e-09
Long-term investments             3.762e-11   3.94e-1

In [24]:
selector = SelectKBest(f_classif, k=20)

X_new = selector.fit_transform(X, y)

selected_features = pd.DataFrame(selector.inverse_transform(X_new), 
                                 index=X.index, 
                                 columns=X.columns)
selected_features.head()

# Dropped columns have values of all 0s, so var is 0, drop them
selected_columns = selected_features.columns[selected_features.var() != 0]

# Get the valid dataset with the selected features.
X_final = selected_features[selected_columns]

/Users/danvaldes/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [ 14  20  84 107 108 116 119 120 140 141 181 190 192 194 196 199] are constant.
  UserWarning)
/Users/danvaldes/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/Users/danvaldes/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y, random_state=1)

In [26]:
reg = sm.OLS(y_train, X_train).fit()

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     2019 PRICE VAR [%]   R-squared (uncentered):                   0.051
Model:                            OLS   Adj. R-squared (uncentered):              0.045
Method:                 Least Squares   F-statistic:                              9.002
Date:                Mon, 11 May 2020   Prob (F-statistic):                    3.11e-23
Time:                        20:08:02   Log-Likelihood:                         -15808.
No. Observations:                2879   AIC:                                  3.165e+04
Df Residuals:                    2862   BIC:                                  3.175e+04
Df Model:                          17                                                  
Covariance Type:            nonrobust                                                  
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Preferred Dividends                       1.012e-08   2.57e-08      0.393      0.694   -4.03e-08    6.06e-08
Dividend per Share                           3.2773      0.853      3.840      0.000       1.604       4.951
priceFairValue                           -1.891e-07   6.25e-07     -0.303      0.762   -1.41e-06    1.04e-06
operatingCashFlowPerShare                 -4.79e-06   8.73e-06     -0.549      0.583   -2.19e-05    1.23e-05
cashPerShare                              1.304e-06   1.79e-06      0.730      0.466    -2.2e-06    4.81e-06
payoutRatio                                  0.0653      0.092      0.709      0.478      -0.115       0.246
Revenue per Share                         3.295e-07   1.39e-06      0.237      0.813    -2.4e-06    3.06e-06
Operating Cash Flow per Share             -4.79e-06   8.73e-06     -0.549      0.583   -2.19e-05    1.23e-05
Cash per Share                            1.304e-06   1.79e-06      0.730      0.466    -2.2e-06    4.81e-06
Shareholders Equity per Share             1.371e-06   4.41e-06      0.311      0.756   -7.28e-06       1e-05
Interest Debt per Share                   -1.97e-06   3.71e-06     -0.531      0.596   -9.25e-06    5.31e-06
Dividend Yield                              13.1655     19.254      0.684      0.494     -24.588      50.918
Payout Ratio                                 0.0653      0.092      0.709      0.478      -0.115       0.246
Capex per Share                           -3.54e-06   9.23e-06     -0.383      0.701   -2.16e-05    1.46e-05
3Y Dividend per Share Growth (per Share)    -5.8124      4.911     -1.184      0.237     -15.442       3.817
Sector_Communication Services                5.1259      8.247      0.622      0.534     -11.045      21.297
Sector_Consumer Defensive                    9.1964      5.219      1.762      0.078      -1.038      19.431
Sector_Financial Services                   17.6179      2.834      6.216      0.000      12.060      23.175
Sector_Industrials                          18.0427      2.998      6.018      0.000      12.164      23.922
Sector_Utilities                            21.2998      7.374      2.889      0.004       6.841      35.759
==============================================================================
Omnibus:                     2382.730   Durbin-Watson:                   1.908
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           144791.298
Skew:                           3.507   Prob(JB):                         0.00
Kurtosis:                      37.027   Cond. No.                     2.04e+22
==============================================================================

Wa

# RANDOM FOREST REGRESSION MODELS

In [52]:
y = market["2019 PRICE VAR [%]"]

In [53]:
x = market[cond].drop(columns={'Ticker','Company Name','2019 PRICE VAR [%]','Class','Market Cap Size'})

X = pd.get_dummies(x)

X.head()

,Revenue,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Interest Expense,Earnings before Tax,Income Tax Expense,Net Income - Non-Controlling int,Net Income - Discontinued ops,...,Sector_Communication Services,Sector_Consumer Cyclical,Sector_Consumer Defensive,Sector_Energy,Sector_Financial Services,Sector_Healthcare,Sector_Industrials,Sector_Real Estate,Sector_Technology,Sector_Utilities
0,15926000.0,0.0,15926000.0,0.0,22534000.0,0.0,-10501000.0,375000.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1,185553000.0,132650000.0,52903000.0,0.0,39042000.0,2037000.0,-971000.0,-267000.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
2,267465000.0,179692000.0,87773000.0,49903000.0,48638000.0,1106000.0,-9778000.0,-7632000.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
3,433947000.0,330414000.0,103533000.0,0.0,47755000.0,0.0,55939000.0,13367000.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,330867000.0,121455000.0,209412000.0,0.0,22784000.0,52248000.0,19686000.0,0.0,7516000.0,0.0,...,0,0,0,0,0,0,0,1,0,0


In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [62]:
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train.values.reshape(-1, 1))

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train.values.reshape(-1, 1))
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1))

In [63]:
regr = RandomForestRegressor(max_depth=2, random_state=0)

In [64]:
regr.fit(X_train_scaled, y_train_scaled)

/Users/danvaldes/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/danvaldes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [66]:
regr.score(X_train_scaled, y_train_scaled)

0.09451597177295899

In [48]:
regr.predict(X_train.loc[2256].values.reshape(1,-1))

array([0.804159])